# Assignment 4.1 - Mood Detection with OpenCV

Submitted By: Labbao, Benedick D.<br>
Performed On: 03/21/2024<br>
Submitted On: 03/22/2024

Submitted To: Engr. Roman M. Richard

---

## Dataset

### Data Collection

In [1]:
from time import sleep
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np

raw_dataset_path = 'dataset/faces/'
preprocessed_dataset_path = 'dataset/preprocessed_faces/'
final_dataset_path = 'dataset/edge_detection/'

face_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_eye.xml')

def display_image(image, title=None, conversion=cv2.COLOR_BGR2RGB):
    image = cv2.cvtColor(image, conversion)
    plt.imshow(image)
    plt.xticks([])
    plt.yticks([])
    
    if title is not None:
        plt.title(title)

    plt.show()

def fix_dataset_names(directory, prefix='face_'):
    files = os.listdir(directory)
    counter = 0
    
    for file in files:
        _, ext = os.path.splitext(file)
        new_name = f"{prefix}{counter:04d}{ext}"
        print('fixing', new_name)
        
        os.rename(os.path.join(directory, file), os.path.join(directory, new_name))
        counter += 1

def generate_name(directory):
    files = os.listdir(directory)
    num_files = len(files)
    face_names = 'face_' + str(num_files).zfill(4)
    return face_names + '.png'

def capture(count=1):
    camera = cv2.VideoCapture(0)

    while (count > 0):
        ret, frame = camera.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)

        for (x,y,w,h) in faces:
            image = frame[y:(y+h), x:(x+w)]
            name = generate_name(raw_dataset_path)
            cv2.imwrite(raw_dataset_path + name, image)
            print(f'saving {name}...')

        sleep(1)
        count-=1

In [50]:
fix_dataset_names(raw_dataset_path + 'angry/', 'angry_')

fixing angry_0000.png
fixing angry_0001.png
fixing angry_0002.png
fixing angry_0003.png
fixing angry_0004.png
fixing angry_0005.png
fixing angry_0006.png
fixing angry_0007.png


In [216]:
capture(20)

saving face_0004.png...
saving face_0005.png...
saving face_0006.png...
saving face_0007.png...
saving face_0008.png...
saving face_0009.png...
saving face_0010.png...
saving face_0011.png...
saving face_0012.png...
saving face_0013.png...
saving face_0014.png...
saving face_0015.png...
saving face_0016.png...
saving face_0017.png...
saving face_0018.png...
saving face_0019.png...
saving face_0020.png...
saving face_0021.png...
saving face_0022.png...
saving face_0023.png...


This will take face images at an random interval to create a dataset for facial recognition.

### Image Pre-processing

Remove background by locating eyes and extracting face region.

In [2]:
def extract_face_region(image):
    center_point = lambda p1, p2: (int((p1[0] + p2[0]) / 2)+3, int((p1[1] + p2[1]) / 2))

    if len(image.shape) == 2:
        gray = image
    else:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    eyes = eye_cascade.detectMultiScale(gray, 1.03, 10, 0, (40, 40))

    # Perform eye detection then find center point of left and right eyes
    eyes_center_points = []
    for (x,y,w,h) in eyes:
        eyes_center_points.append(center_point((x,y), (x+w, y+h)))
        # cv2.rectangle(gray, (x, y), (x+w, y+h), (0, 255, 0), 1)
        # display_image(gray)
        
    # Find center betweem right eye's center point and left eye's center point
    if len(eyes_center_points) < 2:
        return image

    eye_center = center_point(eyes_center_points[0], eyes_center_points[1])
    x, y = eye_center

    # add offset to create a bounding box that only contains face region
    x1, y1, x2, y2 = x - 100, y + 140, x + 100, y - 60
    cv2.rectangle(image, (x1, y1), (x2, y2),  (0, 255, 0), 2)

    return image[y2:y1, x1:x2]

def resize(image, scale=0.7):
    w, h = image.shape
    new_image = cv2.resize(image, (int(w*scale), int(h*scale)))
    return new_image

In [3]:
def preprocess_images(src_dir, dst_dir):
    files = os.listdir(src_dir)
    
    for file in files:
        file_name, ext = os.path.splitext(file)
        print("Processing image:", file_name + ext)

        image = cv2.imread(src_dir + file_name + ext)
        image = extract_face_region(image)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(dst_dir + file_name + ext, resize(gray))

Now we created a function to perform preprocessing on all images and move them in a separate folder.

In [349]:
preprocess_images(raw_dataset_path + 'angry/', preprocessed_dataset_path + 'angry/')

Processing image: angry_0000.png
Processing image: angry_0001.png
Processing image: angry_0002.png
Processing image: angry_0003.png
Processing image: angry_0004.png
Processing image: angry_0005.png
Processing image: angry_0006.png
Processing image: angry_0007.png


In [350]:
preprocess_images(raw_dataset_path + 'happy/', preprocessed_dataset_path + 'happy/')

Processing image: happy_0000.png
Processing image: happy_0001.png
Processing image: happy_0002.png
Processing image: happy_0003.png
Processing image: happy_0004.png
Processing image: happy_0005.png
Processing image: happy_0006.png
Processing image: happy_0007.png
Processing image: happy_0008.png
Processing image: happy_0009.png
Processing image: happy_0010.png
Processing image: happy_0011.png
Processing image: happy_0012.png
Processing image: happy_0013.png
Processing image: happy_0014.png
Processing image: happy_0015.png
Processing image: happy_0016.png
Processing image: happy_0017.png
Processing image: happy_0018.png


In [351]:
preprocess_images(raw_dataset_path + 'sad/', preprocessed_dataset_path + 'sad/')

Processing image: sad_0000.png
Processing image: sad_0001.png
Processing image: sad_0002.png
Processing image: sad_0003.png
Processing image: sad_0004.png
Processing image: sad_0005.png
Processing image: sad_0006.png
Processing image: sad_0007.png
Processing image: sad_0008.png
Processing image: sad_0009.png
Processing image: sad_0010.png


In [352]:
preprocess_images(raw_dataset_path + 'neutral/', preprocessed_dataset_path + 'neutral/')

Processing image: neutral_0000.png
Processing image: neutral_0001.png
Processing image: neutral_0002.png
Processing image: neutral_0003.png
Processing image: neutral_0004.png
Processing image: neutral_0005.png
Processing image: neutral_0006.png
Processing image: neutral_0007.png
Processing image: neutral_0008.png
Processing image: neutral_0009.png
Processing image: neutral_0010.png
Processing image: neutral_0011.png


### Image Processing

Using low-level processing which takes edges, texture elements or regions for this activity, I will be using Otsu's method to perform thresholding.

In [4]:
def perform_otsu_thresholding(image):
    _, binary_image = cv2.threshold(image, 100, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)
    return binary_image

In [5]:
def threshold_images(src_dir, dst_dir):
    files = os.listdir(src_dir)
    
    for file in files:
        file_name, ext = os.path.splitext(file)
        print("Processing image:", file_name + ext)

        image = cv2.imread(src_dir + file_name + ext)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        res_image = perform_otsu_thresholding(gray_image)
        cv2.imwrite(dst_dir + file_name + ext, res_image)

In [375]:
threshold_images(preprocessed_dataset_path + 'angry/', final_dataset_path + 'angry/')

Processing image: angry_0000.png
Processing image: angry_0001.png
Processing image: angry_0002.png
Processing image: angry_0003.png
Processing image: angry_0004.png
Processing image: angry_0005.png
Processing image: angry_0006.png
Processing image: angry_0007.png


In [376]:
threshold_images(preprocessed_dataset_path + 'happy/', final_dataset_path + 'happy/')

Processing image: happy_0000.png
Processing image: happy_0001.png
Processing image: happy_0002.png
Processing image: happy_0003.png
Processing image: happy_0004.png
Processing image: happy_0005.png
Processing image: happy_0006.png
Processing image: happy_0007.png
Processing image: happy_0008.png
Processing image: happy_0009.png
Processing image: happy_0010.png
Processing image: happy_0011.png
Processing image: happy_0012.png
Processing image: happy_0013.png
Processing image: happy_0014.png
Processing image: happy_0015.png
Processing image: happy_0016.png
Processing image: happy_0017.png
Processing image: happy_0018.png


In [377]:
threshold_images(preprocessed_dataset_path + 'neutral/', final_dataset_path + 'neutral/')

Processing image: neutral_0000.png
Processing image: neutral_0001.png
Processing image: neutral_0002.png
Processing image: neutral_0003.png
Processing image: neutral_0004.png
Processing image: neutral_0005.png
Processing image: neutral_0006.png
Processing image: neutral_0007.png
Processing image: neutral_0008.png
Processing image: neutral_0009.png
Processing image: neutral_0010.png
Processing image: neutral_0011.png


In [378]:
threshold_images(preprocessed_dataset_path + 'sad/', final_dataset_path + 'sad/')

Processing image: sad_0000.png
Processing image: sad_0001.png
Processing image: sad_0002.png
Processing image: sad_0003.png
Processing image: sad_0004.png
Processing image: sad_0005.png
Processing image: sad_0006.png
Processing image: sad_0007.png
Processing image: sad_0008.png
Processing image: sad_0009.png
Processing image: sad_0010.png


## Models

### Face Recognition Model

I will be using LBPH Algorithm to create a Face Recognition Model to recognize me

In [6]:
import cv2
import os
import numpy as np

In [7]:
def train_face_recognition_model(data_dir):
    recognizer = cv2.face.LBPHFaceRecognizer_create()

    # Prepare training data
    faces = []
    labels = []

    # Recursively walk through the directory and its subdirectories
    for root, _, files in os.walk(data_dir):
        for filename in files:
            # Check if the file has an image extension
            if any(filename.lower().endswith(ext) for ext in ['.png']):
                img = cv2.imread(os.path.join(root, filename), cv2.IMREAD_GRAYSCALE)
                faces.append(img)
                labels.append('Ben')

    # Assign unique integer labels to each person
    label_dict = {label: idx for idx, label in enumerate(np.unique(labels))}
    labels = [label_dict[label] for label in labels]

    # Train the recognizer
    recognizer.train(faces, np.array(labels))

    # Save the trained model
    recognizer.save("face_recognition_model.xml")
    print("Face recognition model trained and saved successfully.")

def test_face_recognition_model(image):
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read("face_recognition_model.xml")
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Use LBPH recognizer to predict the identity of the face in the test image
    label, confidence = recognizer.predict(gray_image)

    return (label, confidence)

In [42]:
train_face_recognition_model(raw_dataset_path)

Face recognition model trained and saved successfully.


In [8]:
image = cv2.imread(final_dataset_path + '/angry/angry_0000.png')
results = test_face_recognition_model(image)

print(results)

(0, 178.66010843692465)


Testing if the model can recognize processed images

In [10]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("face_recognition_model.xml")

# Initialize the camera
try :
    cam = cv2.VideoCapture(0)

    while True:
        ret, frame = cam.read()

        if(frame == None):
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, 1.03, 5)

        # Draw rectangles around the detected faces and recognize them
        for (x, y, w, h) in faces:
            # Recognize the face
            roi_gray = gray[y:y+h, x:x+w]
            processed_image = extract_face_region(roi_gray)
            processed_image = perform_otsu_thresholding(processed_image)
            id_, confidence = recognizer.predict(processed_image)

            # If recognized face belongs to you (adjust confidence threshold as needed)
            if confidence < 70:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                label_text = f"You (ID: {id_}, Confidence: {confidence:.2f})"
                cv2.putText(frame, label_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            else:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                label_text = f"Unknown (ID: {id_}, Confidence: {confidence:.2f})"
                cv2.putText(frame, label_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            # Print ID and confidence
            print(f"ID: {id_}, Confidence: {confidence:.2f}")

        # Display the resulting frame
        cv2.imshow('Face Recognition', frame)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except Exception as e:
    # Release the capture
    print(e)
    cam.release()
    cv2.destroyAllWindows()

SyntaxError: expected ':' (2750152175.py, line 12)

Testing if it can recognize me using actually images from camera

### 

## Output

In [39]:
# This opens a camera and predict the emotion 

import cv2
import numpy as np
# from tensorflow.keras.models import load_model

# Load the model
# model = load_model('emotion_detection_model.h5')

# Load the pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("face_recognition_model.xml")

# Initialize the camera
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around the detected faces and recognize them
    for (x, y, w, h) in faces:
        # Recognize the face
        roi_gray = gray[y:y+h, x:x+w]
        id_, confidence = recognizer.predict(roi_gray)

        # If recognized face belongs to you (adjust confidence threshold as needed)
        if confidence < 70:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            label_text = f"You (ID: {id_}, Confidence: {confidence:.2f})"
            cv2.putText(frame, label_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            label_text = f"Unknown (ID: {id_}, Confidence: {confidence:.2f})"
            cv2.putText(frame, label_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        # Print ID and confidence
        print(f"ID: {id_}, Confidence: {confidence:.2f}")

    # Display the resulting frame
    cv2.imshow('Emotion Detection', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

ID: 0, Confidence: 71.92
ID: 0, Confidence: 75.96
ID: 0, Confidence: 65.34
ID: 0, Confidence: 58.86
ID: 0, Confidence: 53.49
ID: 0, Confidence: 54.57
ID: 0, Confidence: 53.53
ID: 0, Confidence: 57.57
ID: 0, Confidence: 54.46
ID: 0, Confidence: 54.46
ID: 0, Confidence: 54.60
ID: 0, Confidence: 54.60
ID: 0, Confidence: 56.81
ID: 0, Confidence: 64.34
ID: 0, Confidence: 65.09
ID: 0, Confidence: 56.14
ID: 0, Confidence: 55.01
ID: 0, Confidence: 56.04
ID: 0, Confidence: 58.28
ID: 0, Confidence: 60.77
ID: 0, Confidence: 63.85
ID: 0, Confidence: 56.25
ID: 0, Confidence: 57.02
ID: 0, Confidence: 56.02
ID: 0, Confidence: 55.79
ID: 0, Confidence: 55.90
ID: 0, Confidence: 59.30
ID: 0, Confidence: 58.61
ID: 0, Confidence: 62.86
ID: 0, Confidence: 63.81
ID: 0, Confidence: 67.16
ID: 0, Confidence: 67.66
ID: 0, Confidence: 67.52
ID: 0, Confidence: 70.70
ID: 0, Confidence: 60.93
ID: 0, Confidence: 55.03
ID: 0, Confidence: 55.85
ID: 0, Confidence: 54.10
ID: 0, Confidence: 55.19
ID: 0, Confidence: 53.52
